## load logistic and lorenz logs

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [39]:
# fig = plt.figure(figsize=(20,3.5))
# ax = plt.gca()

def plot_ts(pred):
    y_true, y_pred =pred
    ax = fig.add_subplot(1, 4, i+1)
    ax.plot(y_true[:100,seq_id],'g')
    ax.set_ylim([-20,20])
    ax = fig.add_subplot(2, 4, i+1)
    ax.plot(y_pred[:100,seq_id], 'r')
    ax.set_ylim([-20,20])
    ax.set_title(model)

### load logs for lorenz and print test error

In [3]:
time_stamp ="08-24-17-23-36-53"
exp='traffic_uci'
models =['phased_lstm']#["basic_rnn","basic_lstm","matrix_rnn","tensor_rnn"]
hz_list= [32, 64, 128, 256]
lr_list =[1,2,3,4]


import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt



def read_config(fstream):
    d = {}
    for line in fstream:
        line.rstrip()
        tokens = line.split('\t')
        for token in tokens:
            try:
                key, val = token.split(":")
                d[key] = float(val)                
            except ValueError:
                'error prop'
    return d

def load_log(hz, lr, model):
    data_path = "/var/tmp/tensorRNN/log/"+exp+"/"+time_stamp+"/hz_"+str(hz)+"/"+"lr_1e-"+str(lr)+"/"+model+"/"
    f_pred= "predict.npy"
    f_err = "config_error.out"
    pred =  np.load(data_path+f_pred )
    with open(data_path+"/"+f_err,'r') as fstream:
        config = read_config(fstream)
    return config, pred

errs = np.ones((len(hz_list), len(lr_list), len(models)))
i=0
for hz in hz_list:
    j=0
    for lr in lr_list:
        k=0
        for model in models:
            print('load model {:s}, hidden size {:d}, learning rate 1e-{:d}'.format(model, hz, lr))
            config, pred = load_log(hz, lr, model)
            print('train error:',config['train_error'],'valid error:',config['valid_error'],'test err:',config['test_error'])
            errs[i,j,k] =  config['test_error']
            k +=1
        j+=1

    i+=1

load model phased_lstm, hidden size 32, learning rate 1e-1
train error: 0.0666334535888 valid error: 0.0629022839206 test err: 0.0634530523668
load model phased_lstm, hidden size 32, learning rate 1e-2
train error: 0.064218395853 valid error: 0.0610823549881 test err: 0.0611290274212
load model phased_lstm, hidden size 32, learning rate 1e-3
train error: 0.0671160686177 valid error: 0.0636726009658 test err: 0.0654946523342
load model phased_lstm, hidden size 32, learning rate 1e-4
train error: 0.129395608013 valid error: 0.126906792182 test err: 0.105089335216
load model phased_lstm, hidden size 64, learning rate 1e-1
train error: 0.0677855613446 valid error: 0.0651852338048 test err: 0.065485479904
load model phased_lstm, hidden size 64, learning rate 1e-2
train error: 0.0655154684239 valid error: 0.0623087467617 test err: 0.0634029915749
load model phased_lstm, hidden size 64, learning rate 1e-3
train error: 0.0663142645245 valid error: 0.0632882828837 test err: 0.0603891893869
load

FileNotFoundError: [Errno 2] No such file or directory: '/var/tmp/tensorRNN/log/traffic_uci/08-24-17-23-36-53/hz_256/lr_1e-3/phased_lstm/predict.npy'

In [4]:
np.set_printoptions(precision=3)
for k in range(len(models)):
    print(np.min(errs[:,:,k])) #row min--hidden size
    i,j = np.unravel_index(np.argmin(errs[:,:,k]),(4,4))
    print('hz', hz_list[i], 'lr', lr_list[j])

0.054023356006
hz 128 lr 2
